In [5]:
import pandas as pd
import arrow 
from tqdm import tqdm

In [6]:
data = pd.read_csv('orders.csv')

In [8]:
d0 = data.copy()

In [9]:
# holidays_list = pd.Series(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08', '2018-02-23',
#                  '2018-03-08', '2018-03-09', '2018-04-30', '2018-05-01', '2018-05-02', '2018-05-09', '2018-06-11',
#                  '2018-06-12', '2018-11-05', '2018-12-31', '2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
#                  '2019-01-07', '2019-01-08', '2019-03-08', '2019-05-01', '2019-05-02', '2019-05-03', '2019-05-09',
#                  '2019-05-10', '2019-06-12', '2019-11-04', '2019-12-31', '2020-01-01', '2020-01-02', '2020-01-03',
#                  '2020-01-06', '2020-01-07', '2020-01-08', '2020-02-24', '2020-03-09', '2020-05-01', '2020-05-04',
#                  '2020-05-05', '2020-05-11', '2020-06-12', '2020-11-04', '2020-12-31', '2021-01-01', '2021-01-04',
#                  '2021-01-05', '2021-01-06', '2021-01-07', '2021-01-08', '2021-02-22', '2021-02-23', '2021-03-08',
#                  '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-10', '2021-06-14', '2021-11-04', '2021-11-05',
#                  '2021-12-31', '2022-01-03', '2022-01-04', '2022-01-05', '2022-01-06', '2022-01-07', '2022-02-23',
#                  '2022-03-07', '2022-03-08', '2022-05-02', '2022-05-03', '2022-05-09', '2022-05-10', '2022-06-13',
#                  '2022-11-04'])
# holidays_list = holidays_list.apply(lambda x: arrow.get(x).format('MM-DD'))

In [10]:
# d0['date']=d0['date'].apply(lambda x: arrow.get(x).datetime)
d0['date']=pd.to_datetime(d0['date'])
d0['hour'] = d0.date.dt.hour
d0['weekday'] = d0.date.dt.weekday.astype(int)
# d0['isHoliday'] = d0.date.apply(lambda x: arrow.get(x).format('MM-DD')).isin(holidays_list)*1
# Filtered_holidays = holidays_list.apply(lambda x: arrow.get('2000-'+x).datetime)
# d0['beforeHoliday'] = d0.date.apply(lambda x:abs(Filtered_holidays-arrow.get('2000-'+arrow.get(x).format('MM-DD')).datetime).dt.days.min())

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
import pandas as pd 
from sklearn.metrics import *
# from catboost import CatBoostClassifier, CatBoostRegressor
import numpy as np
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
warnings.filterwarnings('ignore')
import pickle

In [25]:
# d_train = d0.loc[d0.weekday==1]
# X_train, X_test, y_train, y_test = train_test_split(d_train.drop(['date','orders_cnt','delivery_area_id', 'weekday'], axis=1), d_train['orders_cnt'], test_size=0.33, random_state=42)
# params = {'n_estimators': 4000, 'verbose': -1}
# params_fit = {'X': X_train, 'y': y_train, 'eval_set': (X_test, y_test), 'verbose': False, 'early_stopping_rounds':5}
# gbm = lgb.LGBMRegressor(metric='mape', num_iterations=200, **params)
# gbm.fit(**params_fit)
# y_pred0 = gbm.predict(X_test)
# MAPE_t0 = mean_absolute_percentage_error(y_test, y_pred0)
# print(MAPE_t0)

1.0971151039580758


In [27]:
# y_pred0 = gbm.predict(X_test)
# MAPE_t0 = mean_absolute_percentage_error(y_test, y_pred0)

In [29]:
# print('CatBoost test MAPE:')
# # predict_and_evaluate(model_catboost, X_test, y_test)

CatBoost test MAPE:


In [33]:
# pred_df = pd.DataFrame()
# pred_df[str(i)+'_hour'] = pd.Series(range(8,24))
# pred_df[str(i)+'_pred'] = gbm.predict(pred_df)

In [37]:
mon_sum_pred = pd.DataFrame()
for k in range(0, 7):
    locals()['sum_pred'+str(k)]=pd.DataFrame()
    locals()['MAPE'+str(k)]=pd.DataFrame()
    for i in range(8, 24):
            mon_sum_pred.loc[i, 'hour']=int(i)
    for j in tqdm(data.delivery_area_id.unique()):
        d_temp = d0.loc[d0.delivery_area_id==j]
        d_train = d_temp.loc[d0.weekday==k]
        X_train, X_test, y_train, y_test = train_test_split(d_train.drop(['date','orders_cnt','delivery_area_id', 'weekday'], axis=1), d_train['orders_cnt'], test_size=0.33, random_state=42)
    #     X_train = d_train.drop(['date','orders_cnt','delivery_area_id', 'weekday'], axis=1)
    #     y_train = d_train['orders_cnt']
        params = {'n_estimators': 2000, 'verbose': -1}
        params_fit = {'X': X_train, 'y': y_train, 'eval_set': (X_test, y_test), 'verbose': False, 'early_stopping_rounds':5}
        gbm = lgb.LGBMRegressor(metric='mape', **params)
        gbm.fit(**params_fit)
        locals()['sum_pred'+str(k)]['predict'+str(j)]=gbm.predict(pd.DataFrame(mon_sum_pred.hour))
        y_pred0 = gbm.predict(X_test)
        locals()['MAPE'+str(k)].loc[0,'predict'+str(j)] = mean_absolute_percentage_error(y_test, y_pred0)
#         locals()['MAPE'+str(k)].loc[0,'predict'+str(j)]=gbm.evals_result_['valid_0']['mape'][gbm.best_iteration_ - 1]

100%|████████████████████████████████████████████████████████████████████████████████| 593/593 [01:04<00:00,  9.26it/s]


In [58]:
MAPE6.iloc[0].mean()

0.7823496737041628

In [1479]:
for i in range(0, 7):
    with open('sum_pred'+str(i)+'.pickle', 'wb') as handle:
            pickle.dump(locals()['sum_pred'+str(i)], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1476]:
import pickle

In [39]:
for i in range(0,7):
    with open('sum_pred'+str(i)+'.pickle', 'rb') as handle:
        locals()['sum_pred'+str(i)] = pickle.load(handle)

**Поиск оптимального отношения партнёров к количеству заказов**

In [25]:
delays = pd.read_csv('partners_delays.csv')

In [28]:
delays['date']=delays['dttm']
delays0=delays[:200000].drop('dttm', axis=1)
data0=data
delays0 = delays0.merge(data0[:200000], on='date')[['partners_cnt', 'orders_cnt', 'delay_rate']]

In [29]:
delays0 = delays0[['orders_cnt','partners_cnt','delay_rate']]

In [30]:
delays0['par_d_ord']=delays0.partners_cnt/delays0.orders_cnt

In [31]:
delays0['isLate']=(delays0.delay_rate>0) * 1

In [32]:
delays0 = delays0.loc[delays0.isLate==0]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(delays0.orders_cnt), pd.DataFrame(delays0.par_d_ord), test_size=0.33, random_state=42)

In [34]:
params = {'n_estimators': 2000, 'verbose': -1}
params_fit = {'X': X_train, 'y': y_train, 'eval_set': (X_test, y_test), 'verbose': False, 'early_stopping_rounds':5}
gbm = lgb.LGBMRegressor(**params, metrics='auc')
gbm.fit(**params_fit)

LGBMRegressor(metrics='auc', n_estimators=2000, verbose=-1)

In [35]:
y_pred = gbm.predict(X_test)

In [36]:
y_pred.mean()

0.8479300343449937

In [40]:
for j in tqdm(range(0, 7)):
    locals()['count_wod'+str(j)]=pd.DataFrame()
    train_df = locals()['sum_pred'+str(0)]
    for i in train_df.columns:
        locals()['count_wod'+str(j)][i] = gbm.predict(pd.DataFrame(train_df[i]))

100%|██████████| 7/7 [00:16<00:00,  2.30s/it]


In [41]:
count_wod0

,predict0,predict1,predict2,predict3,predict4,predict5,predict6,predict7,predict8,predict9,...,predict583,predict584,predict585,predict586,predict587,predict588,predict589,predict590,predict591,predict592
0,0.805512,0.809317,0.877595,0.820532,0.797605,0.794060,0.805512,0.813839,0.813839,0.805512,...,0.877595,0.846253,0.820532,0.830197,0.846253,0.846253,0.877595,0.877595,0.846253,0.820532
1,0.813839,0.809317,0.877595,0.846253,0.809317,0.805512,0.809317,0.820532,0.813839,0.809317,...,0.877595,0.846253,0.820532,0.830197,0.846253,0.846253,0.877595,0.877595,0.846253,0.830197
2,0.813839,0.809317,0.877595,0.830197,0.802415,0.805512,0.809317,0.820532,0.813839,0.809317,...,0.846253,0.846253,0.820532,0.830197,0.846253,0.846253,0.877595,0.877595,0.846253,0.830197
3,0.813839,0.809317,0.877595,0.830197,0.802415,0.802415,0.813839,0.820532,0.813839,0.809317,...,0.846253,0.846253,0.820532,0.830197,0.877595,0.846253,0.877595,0.877595,0.846253,0.830197
4,0.820532,0.809317,0.877595,0.830197,0.809317,0.802415,0.820532,0.820532,0.813839,0.805512,...,0.846253,0.846253,0.820532,0.830197,0.846253,0.846253,0.877595,0.877595,0.846253,0.830197
5,0.813839,0.809317,0.877595,0.820532,0.809317,0.805512,0.809317,0.820532,0.813839,0.809317,...,0.846253,0.846253,0.813839,0.830197,0.846253,0.846253,0.877595,0.877595,0.846253,0.846253
6,0.820532,0.809317,0.877595,0.820532,0.809317,0.805512,0.813839,0.820532,0.813839,0.809317,...,0.846253,0.846253,0.813839,0.830197,0.846253,0.846253,0.877595,0.877595,0.846253,0.846253
7,0.820532,0.809317,0.877595,0.830197,0.813839,0.809317,0.813839,0.820532,0.813839,0.809317,...,0.846253,0.846253,0.813839,0.830197,0.877595,0.846253,0.877595,0.877595,0.846253,0.846253
8,0.813839,0.809317,0.877595,0.830197,0.809317,0.805512,0.805512,0.813839,0.813839,0.802415,...,0.877595,0.846253,0.813839,0.830197,0.877595,0.846253,0.877595,0.877595,0.846253,0.846253
9,0.813839,0.809317,0.877595,0.820532,0.802415,0.799890,0.805512,0.813839,0.813839,0.797605,...,0.877595,0.846253,0.813839,0.830197,0.846253,0.846253,0.877595,0.877595,0.846253,0.830197


In [42]:
for i in range(0, 7):
    with open('count_wod'+str(i)+'.pickle', 'wb') as handle:
            pickle.dump(locals()['count_wod'+str(i)], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1396]:
# for i in range(0, 7):
#     locals()['count_part'+str(i)] = locals()['count_wod'+str(i)]*locals()['sum_pred'+str(i)]

In [1448]:
for i in range(0, 7):
    locals()['count_part'+str(i)] = y_pred.mean()*locals()['sum_pred'+str(i)]

In [1449]:
sum_pred0

,predict0,predict1,predict2,predict3,predict4,predict5,predict6,predict7,predict8,predict9,...,predict583,predict584,predict585,predict586,predict587,predict588,predict589,predict590,predict591,predict592
0,7.800585,6.661990,2.0,5.409244,11.107143,13.470054,7.638066,5.523676,6.109854,8.333333,...,2.433333,3.371236,5.326311,3.904549,2.758397,2.832500,1.984930,1.992959,3.078947,5.340425
1,5.630272,6.510638,2.0,2.870834,7.000000,7.783613,6.591894,5.434446,5.973490,6.739130,...,2.433333,3.371236,5.326311,3.904549,2.758397,2.832500,1.984930,1.992959,3.078947,4.290813
2,5.757804,6.653972,2.0,3.527226,9.043478,8.080523,6.679198,5.437010,6.009854,7.300000,...,2.536559,3.323833,4.762564,3.865395,2.758397,2.791111,1.984930,1.931940,3.078947,4.270480
3,5.905831,6.701114,2.0,4.176728,8.666667,8.791039,5.609157,5.457523,6.028036,6.727273,...,2.536559,3.323833,4.762564,3.865395,2.496682,2.791111,1.984930,1.744294,3.078947,4.230054
4,5.339465,6.581009,2.0,3.885656,7.423077,9.038971,5.438358,5.457523,5.939021,8.208333,...,2.536559,3.342664,4.629235,3.845289,2.700934,2.791111,2.004264,1.744294,3.078947,3.727570
5,5.962468,6.564805,2.0,4.605495,7.000000,7.911334,6.596663,5.445343,6.040289,7.500000,...,2.502222,3.342664,5.701514,3.845289,2.944804,2.862308,2.004264,1.711422,3.078947,3.282492
6,4.896283,6.545638,2.0,4.799727,7.260870,7.633811,6.372525,5.442701,5.931593,7.280000,...,2.502222,3.342664,5.876376,3.974919,2.713066,2.862308,2.004264,1.617069,3.078947,2.860942
7,5.378571,6.578931,2.0,3.833857,6.444444,7.142340,5.857311,5.442701,5.974854,6.772727,...,2.502222,3.337469,5.932963,3.927697,2.400455,2.795000,2.004264,1.617069,3.078947,2.780146
8,5.792558,6.578931,2.0,3.989629,7.304348,8.261462,8.097524,5.603676,6.081649,8.681818,...,2.486458,3.337469,5.654791,3.927697,2.447124,2.795000,2.072543,1.828499,3.078947,3.082377
9,5.787409,6.593972,2.0,5.252694,9.300000,10.034258,8.347229,5.516176,6.081649,11.359998,...,2.486458,3.425437,5.654791,3.977483,3.114524,2.795000,2.072543,1.828499,3.078947,3.725277


In [1459]:
for i in range(0, 7):
    locals()['count_part'+str(i)+'r'] = count_part0.round(0).astype(int)

In [1464]:
count_part0r

,predict0,predict1,predict2,predict3,predict4,predict5,predict6,predict7,predict8,predict9,...,predict583,predict584,predict585,predict586,predict587,predict588,predict589,predict590,predict591,predict592
0,7,6,2,5,9,11,6,5,5,7,...,2,3,5,3,2,2,2,2,3,5
1,5,6,2,2,6,7,6,5,5,6,...,2,3,5,3,2,2,2,2,3,4
2,5,6,2,3,8,7,6,5,5,6,...,2,3,4,3,2,2,2,2,3,4
3,5,6,2,4,7,7,5,5,5,6,...,2,3,4,3,2,2,2,1,3,4
4,5,6,2,3,6,8,5,5,5,7,...,2,3,4,3,2,2,2,1,3,3
5,5,6,2,4,6,7,6,5,5,6,...,2,3,5,3,2,2,2,1,3,3
6,4,6,2,4,6,6,5,5,5,6,...,2,3,5,3,2,2,2,1,3,2
7,5,6,2,3,5,6,5,5,5,6,...,2,3,5,3,2,2,2,1,3,2
8,5,6,2,3,6,7,7,5,5,7,...,2,3,5,3,2,2,2,2,3,3
9,5,6,2,4,8,9,7,5,5,10,...,2,3,5,3,3,2,2,2,3,3
